In [4]:
try:
    import RPi.GPIO as GPIO
except:
    import Mock.GPIO as GPIO

import time as thyme
from datetime import time, timedelta
from whendo.sdk.client import Client
import whendo.core.util as util
from whendo.core.util import DateTime, DateTime2, Now
import whendo.core.actions.file_action as file_x
import whendo.core.actions.list_action as list_x
import whendo.core.schedulers.timed_scheduler as timed_x
import whendo_gpio.scheduler as sched_x
from whendo.core.scheduler import Immediately
import whendo.core.programs.simple_program as prog_x
import whendo_gpio.action as gpio_x
import whendo.core.actions.sys_action as sys_x
import whendo.core.actions.http_action as http_x

"""
define the actions
"""
gpio_clear = gpio_x.CleanupPins()
file_heartbeat1 = file_x.FileAppend(file="gpio_beat1.txt")
file_heartbeat2 = file_x.FileAppend(file="gpio_beat2.txt")
file_heartbeat3 = file_x.FileAppend(file="gpio_beat3.txt")
system_info = sys_x.SysInfo()
mini_info = sys_x.MiniInfo()
success = list_x.Success()
pause1 = sys_x.Pause()
pause2 = sys_x.Pause(seconds=2)
file_append = file_x.FileAppend(mode="d", file="boomerang.txt")
execute_action = http_x.ExecuteAction(host="192.168.0.26", port=8000, action_name="file_append")
raise_if_equal_to_0 = list_x.RaiseIfEqual(value=0)
raise_if_equal_to_1 = list_x.RaiseIfEqual(value=1)
boomerang = list_x.All(actions=[system_info, execute_action])

green_pin = 27
green_on = gpio_x.SetPin(pin=green_pin, on=True)
green_off = gpio_x.SetPin(pin=green_pin, on=False)
green_toggle = gpio_x.TogglePin(pin=green_pin)
green_state = gpio_x.PinState(pin=green_pin)
green_change_append = file_x.FileAppend(mode="d", file="green_changes.txt")
green_pin_changes = list_x.All(actions=[mini_info, green_state, green_change_append])
green_boomerang = list_x.All(actions=[mini_info, green_state, execute_action])

red_pin = 25
red_on = gpio_x.SetPin(pin=red_pin, on=True)
red_off = gpio_x.SetPin(pin=red_pin, on=False)
red_toggle = gpio_x.TogglePin(pin=red_pin)
red_state = gpio_x.PinState(pin=red_pin)
red_change_append = file_x.FileAppend(mode="d", file="red_changes.txt")
red_pin_changes = list_x.All(actions=[mini_info, red_state, red_change_append])
red_boomerang = list_x.All(actions=[mini_info, red_state, green_state, red_state, execute_action])

toggle_toggle = list_x.All(actions=[green_toggle, red_toggle])
on_on = list_x.All(actions=[green_on, red_on])
off_off = list_x.All(actions=[green_off, red_off])
start_pivot = list_x.All(actions=[red_on, green_on, pause1, green_off])

"""
define the schedulers
"""
often = timed_x.Timely(interval=1)
morning, evening = time(6,0,0), time(18,0,0)
daily_often = timed_x.Timely(start=morning, stop=evening, interval=1)
nightly_often = timed_x.Timely(start=evening, stop=morning, interval=1)
randomly_often = timed_x.Randomly(time_unit=util.TimeUnit.second, low=2, high=5)
timely = timed_x.Timely(interval=1)
timely_at_00_sec = timed_x.Timely(interval=1, second=00)
timely_at_30_sec = timed_x.Timely(interval=1, second=30)
timely_at_10_sec = timed_x.Timely(interval=1, second=10)
timely_every_5 = timed_x.Timely(interval=5)
timely_every_10 = timed_x.Timely(interval=10)
timely_every_30 = timed_x.Timely(interval=30)
immediately = Immediately()
green_rising_falling = sched_x.PinScheduler(pin=green_pin, event=GPIO.BOTH)
red_rising_falling = sched_x.PinScheduler(pin=red_pin, event=GPIO.BOTH)

"""
define the programs
"""
pivot_program = prog_x.PBEProgram().prologue("start_pivot").epilogue("gpio_clear").body_element("timely_every_10", "red_on")
pivot_program2 = prog_x.PBEProgram().prologue("green_on").epilogue("gpio_clear").body_element("timely", "red_toggle")
file_heartbeat_program = prog_x.PBEProgram().prologue("file_heartbeat1").epilogue("file_heartbeat3").body_element("timely", "file_heartbeat2")
toggle_program = prog_x.PBEProgram().body_element("timely_every_5", "toggle_toggle").epilogue("gpio_clear")

green_boomerang_program = prog_x.PBEProgram().body_element("timely_every_5", "green_boomerang")
green_toggle_program = prog_x.PBEProgram().body_element("timely", "green_toggle")
green_monitor_program = prog_x.PBEProgram().body_element("timely_every_5", "green_pin_changes")

red_boomerang_program = prog_x.PBEProgram().body_element("timely_every_5", "red_boomerang")
red_toggle_program = prog_x.PBEProgram().body_element("timely", "red_toggle")
red_monitor_program = prog_x.PBEProgram().body_element("timely_every_5", "red_pin_changes")


def add_actions(client:Client):
    """
    add actions to the 'client' dispatcher
    """
    [ client.add_action(*action) for action in [
        ('gpio_clear', gpio_clear),
        ('toggle_toggle', toggle_toggle),
        ('on_on', on_on),
        ('off_off', off_off),
        ('file_heartbeat1', file_heartbeat1),
        ('file_heartbeat2', file_heartbeat2),
        ('file_heartbeat3', file_heartbeat3),
        ('sys_info', system_info),
        ('success', success),
        ('pause1', pause1),
        ('pause2', pause2),
        ('start_pivot', start_pivot),
        ('execute_action', execute_action),
        ('boomerang', boomerang),
        ('file_append', file_append),
        ('raise_if_equal_to_0', raise_if_equal_to_0),
        ('raise_if_equal_to_1', raise_if_equal_to_1),

        ('green_on', green_on),
        ('green_off', green_off),
        ('green_toggle', green_toggle),
        ('green_state', green_state),
        ('green_pin_changes', green_pin_changes),
        ('green_boomerang', green_boomerang),

        ('red_on', red_on),
        ('red_off', red_off),
        ('red_toggle', red_toggle),
        ('red_state', red_state),
        ('red_pin_changes', red_pin_changes),
        ('red_boomerang', red_boomerang),
        ]
    ]

def add_schedulers(client:Client):
    """
    add schedulers to the 'client' dispatcher
    """
    [ client.add_scheduler(*scheduler) for scheduler in [
        ('often', often),
        ('daily_often', daily_often),
        ('nightly_often', nightly_often),
        ('randomly_often', randomly_often),
        ('timely', timely),
        ('timely_at_00_sec', timely_at_00_sec),
        ('timely_at_10_sec', timely_at_10_sec),
        ('timely_at_30_sec', timely_at_30_sec),
        ('timely_every_5', timely_every_5),
        ('timely_every_10', timely_every_10),
        ('timely_every_30', timely_every_30),
        ('heartbeat', timely_every_30),
        ('immediately', immediately),
        ('green_rising_falling', green_rising_falling),
        ('red_rising_falling', red_rising_falling)
        ]
    ]

def add_programs(client:Client):
    """
    add programs to the 'client' dispatcher
    """
    [ client.add_program(*program) for program in [
        ('pivot_program', pivot_program),
        ('pivot_program2', pivot_program2),
        ('file_heartbeat_program', file_heartbeat_program),
        ('toggle_program', toggle_program),

        ('green_boomerang_program', green_boomerang_program),
        ('green_toggle_program', green_toggle_program),
        ('green_monitor_program', green_monitor_program),

        ('red_boomerang_program', red_boomerang_program),
        ('red_toggle_program', red_toggle_program),
        ('red_monitor_program', red_monitor_program)
        ]
    ]

def schedule_actions(client:Client):
    """
    schedule the actions of interest
    """
    def toggle(datetime2:DateTime2):
        client.schedule_program("green_toggle_program", datetime2=datetime2)
        client.schedule_program("red_toggle_program", datetime2=datetime2)
    def local_monitor(datetime2:DateTime2):
        client.schedule_program("green_monitor_program", datetime2=datetime2)
        client.schedule_program("red_monitor_program", datetime2=datetime2)
    def remote_monitor(datetime2:DateTime2):
        client.schedule_program("green_boomerang_program", datetime2=datetime2)
        client.schedule_program("red_boomerang_program", datetime2=datetime2)
    def event_callback(datetime2:DateTime2):
        client.schedule_action("green_rising_falling", "green_pin_changes")
        client.schedule_action("red_rising_falling", "red_pin_changes")
        client.expire_action("green_rising_falling", "green_pin_changes", DateTime(dt=datetime2.dt2))
        client.expire_action("red_rising_falling", "red_pin_changes", DateTime(dt=datetime2.dt2))
    def pivot(datetime2:DateTime2):
        client.schedule_program("pivot_program", datetime2=datetime2)
    def pivot2(datetime2:DateTime2):
        client.schedule_program("pivot_program2", datetime2=datetime2)
    def file_heartbeat(datetime2:DateTime2):
        client.schedule_program("file_heartbeat_program", datetime2=datetime2)
    def old_school(datetime2:DateTime2):
        [ client.schedule_action(*stuff) for stuff in [
            ('randomly_often', 'red_toggle'),
            ('daily_often', 'green_toggle'),
            ('nightly_often', 'green_toggle')
            ]
        ]
    now = Now.dt()
    datetime2 = DateTime2(dt1=now, dt2=now + timedelta(seconds=90))
    for something in [pivot, remote_monitor]:
        something(datetime2)


def unschedule_schedulers(client):
    """
    unschedule the schedulers
    """
    [ client.unschedule_scheduler(scheduler) for scheduler in [ 
        'randomly_often', 'daily_often', 'nightly_often', 'often'
        ]
    ]
def all_on(client):
    client.execute_action('green_on')
    client.execute_action('red_on')

def all_off(client):
    client.execute_action('green_off')
    client.execute_action('red_off')

def sys_info(client):
    print(client)
    util.PP.pprint(client.execute_action('sys_info'))

def clear_gpio(client):
    client.execute_action("gpio_clear")

"""
set up clients
"""
local = Client(host="192.168.0.26")
pi4 = Client(host='192.168.0.45')
pi3 = Client(host='192.168.0.46')

"""
convenience functions applied to a list of Clients
"""
def stop_init_run(client_list):
    stop(client_list)
    init(client_list)
    run(client_list)
def stop(client_list):
    for c in client_list:
        c.stop_jobs()
def init(client_list):
    for c in client_list:
        clear_gpio(c)
        c.clear_dispatcher()
        add_actions(c)
        add_schedulers(c)
        add_programs(c)
        schedule_actions(c)
def run(client_list):
    for c in client_list:
        c.run_jobs()
def pprint(client_list):
    for c in client_list:
        print(f"job count={c.job_count()} for ({c})")
        c.load_dispatcher().pprint()
def describe(client_list):
    for c in client_list:
        util.PP.pprint(c.describe_all())
def on(client_list):
    for c in client_list:
        all_on(c)
def off(client_list):
    for c in client_list:
        all_off(c)
def info(client_list):
    for c in client_list:
        sys_info(c)
def unschedule(client_list):
    for c in client_list:
        unschedule_schedulers(c)
def gpio_clr(client_list):
    for c in client_list:
        clear_gpio(c)


In [ ]:
pprint([pi3, pi4])

In [5]:
stop_init_run([pi3, pi4])

TypeError: 'PBEProgram' object is not callable

In [ ]:
init([pi3, pi4])

In [ ]:
run([pi3, pi4])

In [ ]:
stop([pi3, pi4])

In [ ]:
info([pi3, pi4])

In [ ]:
on([pi3, pi4])

In [ ]:
off([pi3, pi4])

In [ ]:
describe([pi3,pi4])

In [ ]:
unschedule([pi3,pi4])

In [ ]:
gpio_clr([pi3, pi4])

In [ ]:
!echo $PATH

In [ ]:
!type python

In [ ]:
!jupyter kernelspec list

In [ ]:
local.add_action("file_append", file_append)

In [ ]:
local.load_dispatcher()